In [9]:
import jieba
import jieba.posseg as pseg
#用于中文分词的库
from py2neo import Graph
#用于中文备注的库

In [10]:
#用于导入打标的文件
def dictPath(dict_path = []):
    for p in dict_path:
        jieba.load_userdict(p)

In [11]:
# 导入打标文件
dictPath(['./Data/ngsDict.txt', './Data/nsxDict.txt','./Data/nprDict.txt'])

In [12]:
# 用于分词，把达标过的词从搜索语句中找出来
def get_word_objects(sentence):
    ngs = []
    nsx = []
    npr = []
    for word, tag in pseg.cut(sentence):
        if tag == 'ngs':
            ngs.append(word)
        elif tag == 'nsx':
            nsx.append(word)
        elif tag == 'npr':
            npr.append(word)   
    return ngs, nsx, npr

In [14]:
# 连接neo4j放在文件最前面
url = 'localhost:7474'
# 用户名
username = 'neo4j'
# 密码
password = 'database123'
graph = Graph(url, username=username, password=password)

# 通过一句通用的搜索语句返回所有需要的东西
def searchGraph(label, value):
    rt = graph.run(
        "match (s:`{}`)-[p]->(o) where s.value=~'.*{}.*' return type(p), o.value, labels(o)"
        .format(label,value)).data()
    if len(rt) < 1:
        return -1
    else:
        property_dict = {}
        for po in rt:
            if po['type(p)'] not in property_dict:
                property_dict[po['type(p)']] = [(po['labels(o)'][0], po['o.value']), ]
            else:
                property_dict[po['type(p)']].append((po['labels(o)'][0], po['o.value']))
        return property_dict

In [15]:
# 把neo4j中返回的所有关系进行整理，返回需要的数据
def search(sentence):
    ngs, nsx, npr = get_word_objects(sentence)
    if ngs:
        answ = []
        for i in ngs:
            answer = searchGraph(label='理财产品', value=i)
            ans = []
            if nsx:
                for k,v in answer.items():
                    if k in nsx:
                        a = str(k + ' : ' + v[0][1])
                        ans.append(a)
            else:
                for p,q in answer.items():
                    a = p + ' : ' + q[0][1]
                    ans.append(a)
            answ.append(ans)
        return answ
    elif npr:
        anws = []
        for i in npr:
            lab = i.split('为')[0]
            val = i.split('为')[1]
            answer = searchGraph(label=lab, value=val)
            for j in answer['产品']:
                anws.append(j[1])
        return anws
    else:
        return '请输入想搜索的理财产品全名，或者想要找的属性'

In [22]:
s = input()
a=search(s)

发行机构为徐州铜山农村商业银行股份有限公司的产品


In [21]:
y = input()
b=search(y)

产品状态为预售的产品


In [25]:
a

['金宝理财2020044期',
 '金宝理财2020047期',
 '金宝理财2020051期',
 '金宝理财2020052期',
 '金宝理财2020053期',
 '金宝理财2020048期',
 '金宝理财2020049期',
 '金宝理财2020046期',
 '金宝理财2020045期',
 '金宝理财2020050期',
 '金宝理财2020054期',
 '金宝理财2020055期']

In [26]:
list(set(a).intersection(set(b))) 

['金宝理财2020051期',
 '金宝理财2020052期',
 '金宝理财2020045期',
 '金宝理财2020055期',
 '金宝理财2020048期',
 '金宝理财2020054期',
 '金宝理财2020044期',
 '金宝理财2020053期',
 '金宝理财2020047期',
 '金宝理财2020049期',
 '金宝理财2020046期',
 '金宝理财2020050期']

In [27]:
ino='金宝理财2020051期'
ou='我想要查找金宝理财2020051期'

In [28]:
ino in ou

True

In [69]:
# 不封装写在main里面的写法
sentence = input()
ngs, nsx, npr = get_word_objects(sentence)
if ngs:
    for i in ngs:
        answer = searchGraph(label='理财产品', value=i)
        if nsx:
            for k,v in answer.items():
                if k in nsx:
                    print(k + ' : ' + v[0][1])
        else:
            for p,q in answer.items():
                print(p + ' : ' + q[0][1])
elif npr:
    for i in npr:
        lab = i.split('为')[0]
        val = i.split('为')[1]
        answer = searchGraph(label=lab, value=val)
        for j in answer['产品']:
            print(j[1])
else:
    print('请输入理财产品全名')

金宝理财2020055期的实际天数（天）
认购价格 : 50000.0
机构类别 : 农村合作金融机构
产品销售区域 : 江苏
收益类型 : 保本浮动收益
实际天数（天） : 90
产品状态 : 预售
产品终止日期 : 2020/12/21
产品起始日期 : 2020/09/22
募集结束日期 : 2020/09/15
募集起始日期 : 2020/09/15
风险等级 : 一级(低)
风险等级编号 : 1
募集币种 : 人民币(CNY)
投资性质 : 固定收益类
运作模式 : 封闭式非净值型
募集方式 : 公募
发行机构代码 : C31324
发行机构 : 徐州铜山农村商业银行股份有限公司
业绩比较基准 : 0
期限类型 : 1-3个月(含)
登记编码 : C3132420000049


In [67]:
get_word_objects('实际天数(天)')

([], [], [])